In [1]:
%load_ext autoreload
%autoreload 2

## Importing Custom Modules from Private Repo
from trade.helpers.helper import binomial, implied_vol_bt, time_distance_helper, optionPV_helper,binomial_implied_vol
from trade.assets.rates import get_risk_free_rate_helper
from dbase.DataAPI.ThetaData import retrieve_chain_bulk


Console Logging & File Logging Can be configured using STREAM_LOG_LEVEL and FILE_LOG_LEVEL in environment variables.
Propagate to root logger can be set using PROPAGATE_TO_ROOT_LOGGER in environment variables.
Example:
STREAM_LOG_LEVEL = 'DEBUG'
FILE_LOG_LEVEL = 'INFO'
PROPAGATE_TO_ROOT_LOGGER = 'False'

2025-03-28 18:52:29 trade.helpers.Logging INFO: Logging Root Directory: /Users/chiemelienwanisobi/cloned_repos/QuantFinance/logs
Using Proxy URL: http://18.232.166.224:5500/thetadata


In [2]:
import math
import pandas as pd
import numpy as np
from datetime import datetime

In [3]:
rate_ts = get_risk_free_rate_helper()
rate_ts

Saving to cache from db


,daily,annualized,name,description
Datetime,,,,
2010-01-01,0.000134,0.00050,^IRX,13 WEEK TREASURY BILL
2010-01-04,0.000147,0.00055,^IRX,13 WEEK TREASURY BILL
2010-01-05,0.000160,0.00060,^IRX,13 WEEK TREASURY BILL
2010-01-06,0.000121,0.00045,^IRX,13 WEEK TREASURY BILL
2010-01-07,0.000121,0.00045,^IRX,13 WEEK TREASURY BILL
...,...,...,...,...
2025-03-24,0.004518,0.04182,^IRX,13 WEEK TREASURY BILL
2025-03-25,0.004518,0.04182,^IRX,13 WEEK TREASURY BILL
2025-03-26,0.004522,0.04190,^IRX,13 WEEK TREASURY BILL


## SET UP VARIABLES

In [4]:
## Bulk chain retrieval

aapl = retrieve_chain_bulk('AAPL', None, datetime.today().strftime('%Y-%m-%d'), datetime.today().strftime('%Y-%m-%d'), '16:00')
aapl

,Root,Expiration,Strike,Right,Bid_size,CloseBid,Ask_size,CloseAsk,Date,Midpoint,Weighted_midpoint
datetime,,,,,,,,,,,
2025-03-28,AAPL,2025-05-16,100.0,P,19,0.02,191,0.06,20250328,0.040,0.056381
2025-03-28,AAPL,2025-05-16,180.0,C,7,39.55,7,40.10,20250328,39.825,39.825000
2025-03-28,AAPL,2025-05-16,100.0,C,3,118.20,45,118.85,20250328,118.525,118.809375
2025-03-28,AAPL,2025-05-16,180.0,P,46,1.11,140,1.14,20250328,1.125,1.132581
2025-03-28,AAPL,2025-05-16,265.0,C,79,0.21,22,0.24,20250328,0.225,0.216535
...,...,...,...,...,...,...,...,...,...,...,...
2025-03-28,AAPL,2026-06-18,35.0,P,46,0.10,106,0.18,20250328,0.140,0.155789
2025-03-28,AAPL,2026-06-18,195.0,P,343,13.25,20,13.50,20250328,13.375,13.263774
2025-03-28,AAPL,2026-06-18,195.0,C,346,44.45,19,45.40,20250328,44.925,44.499452


In [5]:
## Pick a strike and expiration

idx = 10
strike = 215.0
strike_df = aapl[aapl.Strike == strike].pivot(index = 'Expiration', columns = 'Right', values = 'Midpoint')
strike, exp, mkt_c, mkt_p = strike, strike_df.index[idx], strike_df['C'].values[idx], strike_df['P'].values[idx]


s0 = 217.85
t = time_distance_helper(exp=exp, strt = datetime.today().strftime('%Y-%m-%d')) ## Expressed in years
r = 0.04188
initial_q = 0.0
actual_q = 0.46/100
t, strike, exp, mkt_c, mkt_p

(0.38329911019849416, 215.0, Timestamp('2025-08-15 00:00:00'), 19.3, 12.35)

## DEFINE NECESSARY FUNCTIONS

In [6]:
def implied_forward_price(k, mkt_c, mkt_p, t, r):
    return (mkt_c - mkt_p) +(k * np.exp(-r * t))

def theoretical_forward_price(s0, t, r, q):
    return s0 * np.exp((r-q) * t)

def implied_forward_growth_rate(k, mkt_c, mkt_p, s0, t, r):
    f_implied = implied_forward_price(k, mkt_c, mkt_p, t, r)
    return f_implied/s0

def idiv_eu_parity(k, mkt_c, mkt_p, s0, t, r):
    implied_growth_factor = implied_forward_growth_rate(k, mkt_c, mkt_p, s0, t, r)
    return -((np.log(implied_growth_factor)/t) )

def naive_idiv_eu_parity(s, k, mkt_c, mkt_p, t, r):
    return s - (k * np.exp(-r * t)) - (mkt_c - mkt_p)


### ***CALCULATE IMPLIED DIVIDEND YIELD***

- We get the initial ***Implied Dividends*** by solving for the Put/Call parity condition below:
    - $ (C(K, S) + P(K, S)) - Ke^{rT} = S^{-qT}$
- After Appropriate reorganizing, we get q as:

    - $q = \frac{1}{T} \cdot (ln(F_{\text{p}}/S))$
    - Where $F_{\text{p}}$ is $ (C(K, S) + P(K, S)) - Ke^{rT} $ known as the prepaid forward

In [7]:
f_implied = implied_forward_price(strike, mkt_c, mkt_p, t, r)
theo_foward = theoretical_forward_price(s0, t, r, initial_q)
implied_growth_factor = implied_forward_growth_rate(strike, mkt_c, mkt_p, s0, t, r)
idiv = idiv_eu_parity(strike, mkt_c, mkt_p, s0, t, r)
print(f"Implied Forward Price: {f_implied:.2f}")
print(f"Theoretical Forward Price: {theo_foward:.2f}")
print(f"Stock Price: {s0:.2f}")
print(f"Implied Forward Growth Rate: {implied_growth_factor:.5f}")
print(f"Implied Dividend Yield: {idiv_eu_parity(strike, mkt_c, mkt_p, s0, t, r):.4%}")
print(f"Total Dividend: ${s0 * (1-np.exp(-idiv*t)):.2f}")
# print(f"Implied Dividend Yield: {idiv_eu_parity(strike, mkt_c, mkt_p, s0, t, r):.4%}")


Implied Forward Price: 218.53
Theoretical Forward Price: 221.38
Stock Price: 217.85
Implied Forward Growth Rate: 1.00310
Implied Dividend Yield: -0.8086%
Total Dividend: $-0.68


In [8]:


call_theo_vol = implied_vol_bt(
    S0 = s0,
    K = strike,
    exp_date = exp,
    r = r,
    y = idiv,
    market_price=mkt_c,
    flag = 'c',
)

put_theo_vol = implied_vol_bt(
    S0 = s0,
    K = strike,
    exp_date = exp,
    r = r,
    y = idiv,
    market_price=mkt_p,
    flag = 'p',
)


eu_call = optionPV_helper(
    spot_price = s0,
    strike_price=strike,
    exp_date = exp,
    risk_free_rate = r,
    dividend_yield = idiv,
    volatility=call_theo_vol,
    putcall = 'c',
    settlement_date_str = datetime.today().strftime('%Y-%m-%d'),
)

eu_put = optionPV_helper(
    spot_price = s0,
    strike_price=strike,
    exp_date = exp,
    risk_free_rate = r,
    dividend_yield = idiv,
    volatility=put_theo_vol,
    putcall = 'p',
    settlement_date_str = datetime.today().strftime('%Y-%m-%d'),
)

print(f"Call Theoretical Volatility: {call_theo_vol:.4f}")
print(f"Put Theoretical Volatility: {put_theo_vol:.4f}")
print(f"BSM Call Theoretical Price: {eu_call:.2f}")
print(f"BSM Put Theoretical Price: {eu_put:.2f}")

Call Theoretical Volatility: 0.2943
Put Theoretical Volatility: 0.2877
BSM Call Theoretical Price: 19.34
BSM Put Theoretical Price: 12.04


In [9]:

strike_df['t'] = strike_df.apply(lambda x: time_distance_helper(x.name, datetime.today().strftime('%Y-%m-%d')), axis = 1)
strike_df = strike_df[strike_df.t > 0]
strike_df['put_vol'] = strike_df.apply(lambda x: implied_vol_bt(s0, strike, r, x.P, x.name, 'p', start = datetime.today(), y = 0.0046), axis = 1)
strike_df['call_vol'] = strike_df.apply(lambda x: implied_vol_bt(s0, strike, r, x.C, x.name, 'c', start = datetime.today(), y = 0.0046), axis = 1)
strike_df['bsm_call'] = strike_df.apply(lambda x: optionPV_helper(s0, strike, x.name, r, 0.0046, x.call_vol, 'c', datetime.today().strftime('%Y-%m-%d')), axis = 1)
strike_df['bsm_put'] = strike_df.apply(lambda x: optionPV_helper(s0, strike, x.name, r, 0.0046, x.put_vol, 'p', datetime.today().strftime('%Y-%m-%d')), axis = 1)
strike_df

Right,C,P,t,put_vol,call_vol,bsm_call,bsm_put
Expiration,,,,,,,
2025-04-04,5.425,2.385,0.019165,0.326355,0.331785,5.646340,2.581006
2025-04-11,6.825,3.600,0.038330,0.304976,0.311219,6.985484,3.726456
2025-04-17,7.700,4.350,0.054757,0.294771,0.299809,7.835971,4.448826
2025-04-25,8.625,4.975,0.076660,0.277362,0.287372,8.728099,5.031526
2025-05-02,10.350,6.600,0.095825,0.311643,0.318944,10.434768,6.627465
2025-05-09,11.175,7.225,0.114990,0.307826,0.316478,11.246047,7.228757
2025-05-16,11.675,7.725,0.134155,0.302667,0.306080,11.734555,7.707615
2025-06-20,14.425,9.650,0.229979,0.285060,0.290619,14.449913,9.542510
2025-07-18,16.300,10.850,0.306639,0.277210,0.284535,16.306576,10.676094


In [10]:
strike_df['idiv'] = strike_df.apply(lambda x: idiv_eu_parity(strike, x.bsm_call, x.bsm_put, s0, time_distance_helper(x.name, datetime.today().strftime('%Y-%m-%d')), r), axis = 1)
strike_df['cash_div'] = strike_df.apply(lambda x: f"${s0 * (1-np.exp(-x.idiv*t)):.2f}", axis = 1)
strike_df['idiv2'] = strike_df.apply(lambda x: naive_idiv_eu_parity(s0, strike, x.C, x.P,x.t, r), axis = 1) 
strike_df

Right,C,P,t,put_vol,call_vol,bsm_call,bsm_put,idiv,cash_div,idiv2
Expiration,,,,,,,,,,
2025-04-04,5.425,2.385,0.019165,0.326355,0.331785,5.646340,2.581006,-0.010260,$-0.86,-0.017504
2025-04-11,6.825,3.600,0.038330,0.304976,0.311219,6.985484,3.726456,-0.007684,$-0.64,-0.030147
2025-04-17,7.700,4.350,0.054757,0.294771,0.299809,7.835971,4.448826,-0.003744,$-0.31,-0.007522
2025-04-25,8.625,4.975,0.076660,0.277362,0.287372,8.728099,5.031526,-0.009423,$-0.79,-0.110846
2025-05-02,10.350,6.600,0.095825,0.311643,0.318944,10.434768,6.627465,-0.004608,$-0.39,-0.038904
2025-05-09,11.175,7.225,0.114990,0.307826,0.316478,11.246047,7.228757,-0.005363,$-0.45,-0.067099
2025-05-16,11.675,7.725,0.134155,0.302667,0.306080,11.734555,7.707615,0.000945,$0.08,0.104569
2025-06-20,14.425,9.650,0.229979,0.285060,0.290619,14.449913,9.542510,0.000069,$0.01,0.135841
2025-07-18,16.300,10.850,0.306639,0.277210,0.284535,16.306576,10.676094,-0.000555,$-0.05,0.143388
